In [24]:
import numpy as np
import pandas as pd
import itertools

import statsmodels.api as sm

In [25]:
rec = pd.read_excel("data/1832_v4.xlsx", index_col=0)
rec = rec[[
    "nom_rue_norm_2",
    "no_maison",
    "proprietaire_nom_norm_2",
    "chef_prenom_norm",
    "chef_nom_norm_2",
    "chef_annee_naissance",
    "epouse_nom_norm",
    "epouse_annee_naissance",
    "enfants_dans_la_commune_prenom_norm",
    "enfants_annee_naissance",
    "chef_origine_norm_2",
    "chef_annee_arrivee",
    "chef_vocation_norm_2",
    "pensionnaires_prenom_norm",
    "pensionnaires_nom_norm",
    "pensionnaires_origine_norm",
    "Page"
]]
rec = rec.rename(columns={
    "nom_rue_norm_2": "nom_rue",
    "proprietaire_nom_norm_2": "proprietaire_nom",
    "chef_prenom_norm": "chef_prenom",
    "chef_nom_norm_2": "chef_nom",
    "epouse_nom_norm": "epouse_nom",
    "enfants_dans_la_commune_prenom_norm": "enfants_dans_la_commune_prenom",
    "chef_origine_norm_2": "chef_origine",
    "chef_vocation_norm_2": "chef_vocation",
    "pensionnaires_prenom_norm": "pensionnaires_prenom",
    "pensionnaires_nom_norm": "pensionnaires_nom",
    "pensionnaires_origine_norm": "pensionnaires_origine",
    "Page" : "page"
})

In [26]:
rec.head()

,nom_rue,no_maison,proprietaire_nom,chef_prenom,chef_nom,chef_annee_naissance,epouse_nom,epouse_annee_naissance,enfants_dans_la_commune_prenom,enfants_annee_naissance,chef_origine,chef_annee_arrivee,chef_vocation,pensionnaires_prenom,pensionnaires_nom,pensionnaires_origine,page
1,marterey,1,weidmann,georges henri,weidmann,1764,·,·,·,·,tolochenaz,1786,charon,louise|anna|francois|jn adam,weidmann|la maurer|kirsner|reverber,tolochenaz|frutigen|cirol|bavarois,1
2,marterey,2,collioud,louis,collioud,17936,blanchoud,1800,marie|henri|charles,1822|1823|1825,rolle,1821,marechal-ferrant,louis|marc,henny|bourgeois,montherod|rolle,1
3,marterey,3,fiaux,louise,piot,1785,neissance,·,benjamin,1815,pailly,nee,lingere,·,·,·,1
4,marterey,·,fiaux,monsieur,de gingins,·,seigneux,·,·,·,gingins,·,rentier,louise|marianne|louis,oswald|thelin|vannod,thoune|mex|orny,1
5,marterey,·,fiaux,louis vincent,lavanchy,1802,baudet,1805,·,·,lutry,ne,garcon voiturier,·,·,·,1


In [27]:
rec.dtypes

nom_rue                           object
no_maison                         object
proprietaire_nom                  object
chef_prenom                       object
chef_nom                          object
chef_annee_naissance              object
epouse_nom                        object
epouse_annee_naissance            object
enfants_dans_la_commune_prenom    object
enfants_annee_naissance           object
chef_origine                      object
chef_annee_arrivee                object
chef_vocation                     object
pensionnaires_prenom              object
pensionnaires_nom                 object
pensionnaires_origine             object
page                               int64
dtype: object

In [28]:
len(rec)

3674

## Linear regression

In this section, we will use the propensity scores to match the treated and control groups. The treated group is the houses living in the same road, whereas the control group is the houses living on different roads.

In [29]:
def clean_year(rec_data, on_column):
    valid_chief_year_birth = rec_data[on_column].unique()
    valid_chief_year_birth = np.array([x for x in valid_chief_year_birth if str(x).isdigit()]).astype(int)
    valid_chief_year_birth = valid_chief_year_birth[(valid_chief_year_birth >= 1732) & (valid_chief_year_birth <= 1832)]
    valid_chief_year_birth = valid_chief_year_birth.astype(str)

    return rec_data[rec_data[on_column].isin(valid_chief_year_birth)]

In [30]:
def clean_chief_origin(rec_data, min_chief_origin_thd=10, invalid_chief_origin=["·"]):
    new_rec_data = rec_data.copy()
    new_rec_data["chef_origine"] = new_rec_data["chef_origine"].str.lower()
    valid_chief_origin = new_rec_data \
        .groupby("chef_origine") \
        .agg({"chef_origine": "count"}) \
        .rename(columns={"chef_origine": "count"}) \
        .to_dict()["count"]
    valid_chief_origin = [k for k, v in valid_chief_origin.items() if v >= min_chief_origin_thd]
    valid_chief_origin = [x for x in valid_chief_origin if x not in invalid_chief_origin]

    return new_rec_data[new_rec_data["chef_origine"].isin(valid_chief_origin)]

In [31]:
def clear_chief_vocation(rec_data, min_chief_origin_thd=10, invalid_chef_vocation=["·"]):
    new_rec_data = rec_data.copy()
    new_rec_data["chef_vocation"] = new_rec_data["chef_vocation"].str.lower()
    valid_chef_vocation = rec_data \
        .groupby("chef_vocation") \
        .agg({"chef_vocation": "count"}) \
        .rename(columns={"chef_vocation": "count"}) \
        .to_dict()["count"]
    valid_chef_vocation = [k for k, v in valid_chef_vocation.items() if v >= min_chief_origin_thd]
    valid_chef_vocation = [x for x in valid_chef_vocation if x not in invalid_chef_vocation]

    return new_rec_data[new_rec_data["chef_vocation"].isin(valid_chef_vocation)]

In [32]:
rec_data = rec[[
    "chef_annee_arrivee",
    "chef_annee_naissance",
    "epouse_annee_naissance",
    "chef_origine",
    "chef_vocation",
    "nom_rue"
]]

rec_data.head()

,chef_annee_arrivee,chef_annee_naissance,epouse_annee_naissance,chef_origine,chef_vocation,nom_rue
1,1786,1764,·,tolochenaz,charon,marterey
2,1821,17936,1800,rolle,marechal-ferrant,marterey
3,nee,1785,·,pailly,lingere,marterey
4,·,·,·,gingins,rentier,marterey
5,ne,1802,1805,lutry,garcon voiturier,marterey


In [33]:
#rec_data = clean_year(rec_data, "chef_annee_arrivee")
rec_data = clean_year(rec_data, "chef_annee_naissance")
rec_data = clean_year(rec_data, "epouse_annee_naissance")
rec_data = clean_chief_origin(rec_data, min_chief_origin_thd=10)
rec_data = clear_chief_vocation(rec_data, min_chief_origin_thd=10)
rec_data.head()

,chef_annee_arrivee,chef_annee_naissance,epouse_annee_naissance,chef_origine,chef_vocation,nom_rue
11,1820,1806,1801,corporation vaudoise,cordonnier,marterey
19,1802,1778,1778,lutry,journalier,marterey
24,·,1778,1780,bottens,vigneron,marterey
35,·,1794,1804,corporation vaudoise,cordonnier,marterey
39,ne,1779,1778,lausanne,charpentier,marterey


In [34]:
len(rec_data)

444

In [35]:
rec_data_pairs = itertools.combinations(rec_data.index, 2)
rec_data_pairs = pd.DataFrame(rec_data_pairs, columns=["id_1", "id_2"])
rec_data_pairs = rec_data_pairs \
        .merge(rec_data, left_on="id_1", right_index=True) \
        .merge(rec_data, left_on="id_2", right_index=True, suffixes=("_1", "_2"))
rec_data_pairs["same_road"] = (rec_data_pairs["nom_rue_1"] == rec_data_pairs["nom_rue_2"]).astype(int)
rec_data_pairs = rec_data_pairs.drop(columns=["id_1", "id_2", "nom_rue_1", "nom_rue_2"])
rec_data_pairs.head()

,chef_annee_arrivee_1,chef_annee_naissance_1,epouse_annee_naissance_1,chef_origine_1,chef_vocation_1,chef_annee_arrivee_2,chef_annee_naissance_2,epouse_annee_naissance_2,chef_origine_2,chef_vocation_2,same_road
0,1820,1806,1801,corporation vaudoise,cordonnier,1802,1778,1778,lutry,journalier,1
1,1820,1806,1801,corporation vaudoise,cordonnier,·,1778,1780,bottens,vigneron,1
443,1802,1778,1778,lutry,journalier,·,1778,1780,bottens,vigneron,1
2,1820,1806,1801,corporation vaudoise,cordonnier,·,1794,1804,corporation vaudoise,cordonnier,1
444,1802,1778,1778,lutry,journalier,·,1794,1804,corporation vaudoise,cordonnier,1


In [36]:
rec_data_pairs["same_chief_origin"] = rec_data_pairs["chef_origine_1"] == rec_data_pairs["chef_origine_2"]
rec_data_pairs["same_chief_origin"] = rec_data_pairs["same_chief_origin"].astype(int)
rec_data_pairs["same_chef_vocation"] = rec_data_pairs["chef_vocation_1"] == rec_data_pairs["chef_vocation_2"]
rec_data_pairs["same_chef_vocation"] = rec_data_pairs["same_chef_vocation"].astype(int)
rec_data_pairs["diff_chief_year_birth"] = np.abs(rec_data_pairs["chef_annee_naissance_1"].astype(int) - rec_data_pairs["chef_annee_naissance_2"].astype(int))
rec_data_pairs["diff_spouse_year_birth"] = np.abs(rec_data_pairs["epouse_annee_naissance_1"].astype(int) - rec_data_pairs["epouse_annee_naissance_2"].astype(int))
rec_data_pairs = rec_data_pairs[[
    "same_road",
    "same_chief_origin",
    "same_chef_vocation",
    "diff_chief_year_birth",
    "diff_spouse_year_birth"
]]
rec_data_pairs.head()

,same_road,same_chief_origin,same_chef_vocation,diff_chief_year_birth,diff_spouse_year_birth
0,1,0,0,28,23
1,1,0,0,28,21
443,1,0,0,0,2
2,1,1,1,12,3
444,1,0,0,16,26


In [37]:
X = rec_data_pairs.drop(columns=["same_road"])
y = rec_data_pairs["same_road"]

X.shape, y.shape

((98346, 4), (98346,))

In [38]:
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              same_road   R-squared (uncentered):                   0.030
Model:                            OLS   Adj. R-squared (uncentered):              0.030
Method:                 Least Squares   F-statistic:                              757.0
Date:                Sun, 05 Mar 2023   Prob (F-statistic):                        0.00
Time:                        13:23:37   Log-Likelihood:                          34848.
No. Observations:               98346   AIC:                                 -6.969e+04
Df Residuals:                   98342   BIC:                                 -6.965e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
same_chief_origin          0.0136      0.002      8.745      0.000       0.011       0.017
same_chef_vocation         0.0596      0.002     28.963      0.000       0.056       0.064
diff_chief_year_birth      0.0007   5.87e-05     11.662      0.000       0.001       0.001
diff_spouse_year_birth     0.0005   6.11e-05      7.578      0.000       0.000       0.001
==============================================================================
Omnibus:                   100698.913   Durbin-Watson:                   0.257
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3697867.339
Skew:                           5.434   Prob(JB):                         0.00
Kurtosis:                      31.005   Cond. No.                         91.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""